# Introduction #

Run this cell to set everything up!

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.time_series.ex5 import *

# Setup notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import (CalendarFourier,
                                           CalendarSeasonality,
                                           CalendarTimeTrend)
from statsmodels.tsa.tsatools import lagmat


# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)


# Utility functions
def add_trend(X, freq="D", order=1):
    base_period = X.index[0]
    trend = CalendarTimeTrend(
        freq=freq, order=order, constant=False, base_period=base_period,
    )
    X = X.join(trend.in_sample(X.index))
    return X


def add_seasonal(X, freq="D", period="W"):
    seasonality = CalendarSeasonality(freq="D", period="W")
    X = X.join(seasonality.in_sample(X.index))
    return X


def add_fourier(X, freq="A", order=52):
    fourier = CalendarFourier(freq=freq, order=order)
    X = X.join(fourier.in_sample(X.index))
    return X


def add_lags(X, y, lags=[1]):
    X_lag = lagmat(y, maxlag=max(lags), use_pandas=True, trim="both")
    X_lag = X_lag.iloc[:, [lag - 1 for lag in lags]]
    X = X.join(X_lag).dropna()
    y = y.loc[X.index]
    return X, y 


# Load 1C data
data_dir = Path("../input/ts-course-data")
df_train = pd.read_csv(data_dir / "1c_train.csv", parse_dates=["date"])

# Aggregate item sales into a single time series
ts_cnt = df_train.pivot_table(index="date", values="item_cnt_day", aggfunc="sum")
ts_cnt = ts_cnt.to_period("D")
ts_cnt = ts_cnt["item_cnt_day"]

ts_valid = ts_cnt.iloc[-90:]
ts_train = ts_cnt.drop(ts_valid.index)

-------------------------------------------------------------------------------

How would you define the forecasting goal in the following scenarios?
1. TODO
2. TODO
3. TODO

# 1) Defining the Forecasting Goal

After you've thought about your answer, run this cell for the solution.

In [ ]:
# View the solution (Run this cell to receive credit!)
q_1.check()

-------------------------------------------------------------------------------

Run this cell to create the data splits for the next question.

In [ ]:
def make_deterministic_features(ts):
    X = pd.DataFrame(index=ts.index)
    X = add_trend(X, order=2)
    X = add_seasonal(X)
    X = add_fourier(X, order=52)
    return X


X = make_deterministic_features(ts_cnt)
X_train = X.loc[ts_train.index, :]
X_valid = X.loc[ts_valid.index, :]

y = ts_cnt.copy()
y_train = y.loc[ts_train.index]
y_valid = y.loc[ts_valid.index]

# 2) Deterministic Forecast

Make a forecast on the validation set using only deterministic features.

In [ ]:
# YOUR CODE HERE
#_UNCOMMENT_IF(PROD)_
#____

# Check your answer
q_2.check()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_2.hint()
#_COMMENT_IF(PROD)_
q_2.solution()

In [ ]:
#%%RM_IF(PROD)%%
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = pd.Series(model.predict(X_train), index=y_train.index)
y_forecast = pd.Series(model.predict(X_valid), index=y_valid.index)

rmse = np.sqrt(np.nanmean((y_valid - y_forecast) ** 2))
print(f"Validation RMSE: {rmse}")

q_2.assert_check_passed()

Run this cell if you'd like to see a plot of your forecast.

In [ ]:
ax = ts_cnt.plot(style=".", color="0.25")
ax = y_pred.plot(ax=ax, color="C0")
y_forecast.plot(ax=ax, color="C3");

-------------------------------------------------------------------------------

Run this cell to create the data splits for the next question.

In [ ]:
X = make_deterministic_features(ts_cnt)
y = ts_cnt.copy()

# 3) Validate with Backtest

Instead of using a validation set, use the backtesting method to test your model.

In [ ]:
n_splits = len(y) // 2
tscv = TimeSeriesSplit(n_splits=n_splits, max_train_size=None)
rmse = (-1) * cross_val_score(
    LinearRegression(), X, y, cv=tscv, scoring="neg_mean_squared_error"
)
rmse = np.sqrt(rmse.mean())
print("Backtest RMSE: ", rmse)

-------------------------------------------------------------------------------

Run this cell to set up the training data for this question.

In [ ]:
X_det = make_deterministic_features(ts_cnt)
X_train = X_det.loc[ts_train.index, :]
y_train = ts_cnt.loc[X_train.index]
X_train, y_train = add_lags(X_train, y_train, lags=[1, 3, 6])

# (Optional) Forecasting with Nondeterministic Features #

Using a (nondeterministic) time series as a feature in a machine learning problem creates a unique challenge: we don't actually know the feature's value when we make the forecast!

There are two ways you can meet this challenge.

- Recursive strategy

<img>animation of recursive strategy</img>

- Direct strategy

<img>animation of direct strategy</img>

Pros and Cons:
- Recursive: error blows up
- Direct: computationally expensive

In this (optional) question, you'll explore making forecasts with the recursive method.

Fitting the model.

In [ ]:
maxlag = 24

# Feature engineering for training data
X = pd.concat([
    get_trend(df.Demand),
    get_fourier_features(df.Demand, period="W", order=14),
    get_lags(df.Demand, maxlag=maxlag),
],
              axis=1)
y = df.Demand.copy()

# Fit regression model
model = LinearRegression()
model.fit(X, y)
y_fit = pd.Series(model.predict(X), index=y.index)  # fitted values

The features for the forecast.

In [ ]:
# 24 hours in a day, 7 days in a week
steps = 24 * 7

# We'll just create the deterministic features. The lags we'll add as
# we step through the forecasts.
X_f = pd.concat([
    get_trend(df.Demand, steps=steps),
    get_fourier_features(df.Demand, period="W", order=14, steps=steps),
],
                axis=1)

Here is the recursive forecast. The idea is fairly straightforward, but it does require a bit of index juggling with Pandas.

In [ ]:
# Create a container for the forecasts. We also need enough of the
# original values to create the starting lags.
z = y.reindex(y.index.append(X_f.index))
z = z.iloc[-(maxlag + steps + 1):, ]

# Join lagged values to the feature set.
X_f = X_f.join(get_lags(z, maxlag))

# Define the forecast origin.
origin = X.index[-1]  # '2008-08-27 23:00'

# At each step, make a forecast. Then, add the forecast's lagged values
# to the feature set.
for s in range(1, steps + 1):
    z[origin + s] = model.predict(X_f.loc[[origin + s]])
    X_f.update(get_lags(z, maxlag), overwrite=False)

# Extract the forecasts
y_pred = z[origin + 1:origin + steps]

Now let's see how we did.

In [ ]:
# Plot the results
_ = y.plot(style='.', color='0.25')
_ = y_fit.plot(color='C0')
_ = y_pred.plot(color='C3')

-------------------------------------------------------------------------------

# Keep Going #